<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Investigaci-n-de-operaciones/blob/main/Tarea3IO/Metodo_simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sympy as sp
from IPython.display import Latex
from IPython.display import display_latex
sp.init_printing()

In [ ]:
class MetodoSimplex:
  #------------------------Método simplex base---------------------------
  def solve(M,sim,H):
    M=np.array(M,dtype="float64")
    #Selecciona el pivote
    ve=np.argmax(M[-1,:-1])
    vs=np.argmin([(i if i>0 else np.inf) for i in M[:-1,-1]/M[:-1,ve]])
    #Hace el remplazo de la variable
    H[vs]=sim[ve]
    M[vs]=M[vs]/M[vs][ve]
    #Hace ceros en la columna del pivote
    for i in range(len(M)):
      #Realiza el procedimiento solo si la fila no es la del pivote
      if i!=vs:
        M[i]=M[i]*M[vs][ve]-M[vs]*M[i][ve]
    [print(i) for i in M.round(2).tolist()]
    print("\n")
    #Verifica si hay numeros positivos en la ultima fila exceptuando el resultado en p
    if len(M[-1,:-1][M[-1,:-1]>0])>0:
      #Si hay positivos mayores a cero llama a la funcion pero con la nueva matriz
      return MetodoSimplex.solve(M,sim,H)
    #a grega a H un valor resultado para devolver un diccionario con los valores
    H.append("Resultado")
    M[-1][-1]=abs(M[-1][-1])
    return {H[i]: M[i][-1] for i in range(len(H))}
  #------------------------Método dual---------------------------
  #Esta funcion permite hacer la division valida del método dual, la cual solo puede hacerse si son de signo opuesto
  def __divsim(a,b):
    k=[]
    for i in range(len(a)):
      if (a[i]>0 and b[i]<0) or (a[i]<0 and b[i]>0):
        k.append(a[i]/b[i])
      else:
        #Como el algoritmo va a pedir el número más grande agrega un menos infinito para asegurar que nunca sea seleccionado
        k.append(-np.inf)
    return np.array(k)

  def dual(M,sim,H):
    M=np.array(M,dtype="float64")
    #Selecciona al pivote
    vs =np.argmin([i for i in M[:-1,-1]])#Toma el minimo de un arreglo clon de la ultima columna de M
    ve=np.argmax(MetodoSimplex.__divsim(M[-1,:-1],M[vs,:-1]))#Selecciona el máximo de la division realizada
    #Hace el remplazo
    H[vs]=sim[ve]
    M[vs]=M[vs]/M[vs][ve]
    #Hace el despeje
    for i in range(len(M)):
      if i!=vs:
        M[i]=M[i]*M[vs][ve]-M[vs]*M[i][ve]
    print(M,"\n")
    #Si hay numeros negativos en la ultima columna vuelve a ejecutar simplex pero con la nueva matriz
    if len(M[:-1,-1][M[:-1,-1]<0])>0:
      return MetodoSimplex.dual(M,sim,H)
    H.append("Resultado")
    M[-1][-1]=abs(M[-1][-1])
    return {H[i]: M[i][-1] for i in range(len(H))}

#Max

Maximizar: $z=4x_1+6x_2$

$$SA\begin{cases}
2x_1+x_2\leq 180\\
x_1+2x_2\leq 160\\
x_1+x_2\leq 100
\end{cases}$$

Al convertirlo en igualdades
$$SA\begin{cases}
2x_1+x_2+h_1= 180\\
x_1+2x_2+h_2= 160\\
x_1+x_2+h_3=100
\end{cases}$$

In [ ]:
sim=['x1','x2','h1','h2','h3']
H=['h1','h2','h3']
M=[[2,1,1,0,0,180],
  [1,2,0,1,0,160],
  [1,1,0,0,1,100],
  [4,6,0,0,0,0]]
MetodoSimplex.solve(M,sim,H)

Maximizar: $z=200x_1+100x_2$
$$SA\begin{cases}
5x_1+3x_2\leq 105\\
2x_1+4x_2\leq 70
\end{cases}$$
Al convertirlo a igualdades:
$$SA\begin{cases}
5x_1+3x_2+h1=105\\
2x_1+4x_2+h2=70
\end{cases}$$

In [ ]:
sim=['x1','x2','h1','h2']
H=['h1','h2']
M=[[5,3,1,0,105],
   [2,4,0,1,70],
   [200,100,0,0,0]]
MetodoSimplex.solve(M,sim,H)

Maximizar: $z=5x_1+2x_2$
$$SA\begin{cases}
3x_1+2x_2\leq 2400\\
x_2\leq 800\\
2x1\leq 1200
\end{cases}$$
Al convertirlo a igualdades:
$$SA\begin{cases}
3x_1+2x_2+h1=2400\\
x_2+h2= 800\\
2x1+h3= 1200
\end{cases}$$

In [ ]:
sim=['x1','x2','h1','h2','h3']
H=['h1','h2','h3']
M=[[3,2,1,0,0,2400],
  [0,1,0,1,0,800],
  [2,0,0,0,1,1200],
  [5,2,0,0,0,0]]
MetodoSimplex.solve(M,sim,H)

#Min

In [ ]:
sim=['x','y','t','u','v']
H=['t','u','v']
M=[[-5,-15,1,0,0,-50],
    [-20,-5,0,1,0,-40],
    [15,2,0,0,1,60],
    [4,2,0,0,0,0]]
MetodoSimplex.dual(M,sim,H)

In [ ]:
sim=['x1','x2','s1','s2','s3','s4']
H=['s1','s2','s3','s4']
M=[[-6,-2,1,0,0,0,-6],
   [ 4, 3,0,1,0,0, 6],
   [-4,-3,0,0,1,0,-6],
   [1 , 2,0,0,0,1, 4],
   [8 , 2,0,0,0,0, 0]]
MetodoSimplex.dual(M,sim,H)

#Método de la gran M

Minimizar: $z=4x_1+x_2$
$$SA\begin{cases}
3x_1+x_2=3\\
4x_1+3x_2\geq 6\\
x_1+2x_2 \leq 4
\end{cases}$$
Al convertirlas a igualdades:
$$SA\begin{cases}
3x_1+x_2+A_1=3\\
4x_1+3x_2+A_2-H_1\geq 6\\
x_1+2x_2+H_2 \leq 4
\end{cases}$$

y la última fila es: $$z+x_1(7M-4)+x_2(4M-1)-H_1M=9M$$

In [ ]:
sim=['x1','x2','a1','a2','h1','h2']
H=['a1','a2','h2']
M=1e2
K=[[3,1,1,0,0,0,3],
   [4,3,0,1,-1,0,6],
   [1,2,0,0,0,1,4],
   [7*M-4,4*M-1,0,0,-M,0,9*M]]
MetodoSimplex.solve(K,sim,H)

Minimizar: $z=6x_1+5x_2$
$$SA\begin{cases}
3x_1+x_2=3\\
4x_1+3x_2\geq 6\\
x_1+2x_2 \leq 4
\end{cases}$$
Al convertirlas a igualdades:
$$SA\begin{cases}
3x_1+x_2+A_1=3\\
4x_1+3x_2+A_2-H_1\geq 6\\
x_1+2x_2+H_2 \leq 4
\end{cases}$$

y la última fila es: $$z+x_1(7M-6)+x_2(4M-5)-H_1M=9M$$

In [ ]:
sim=['x1','x2','a1','a2','h1','h2']
H=['a1','a2','h2']
M=1e2
K=[[3,1,1,0,0,0,3],
   [4,3,0,1,-1,0,6],
   [1,2,0,0,0,1,4],
   [7*M-6,4*M-5,0,0,-M,0,9*M]]
MetodoSimplex.solve(K,sim,H)

Minimizar: $z=4x_1+2x_2$

$$SA\begin{cases}
    5x_1+15x_2\geq 50\\
    20x_1+5x_2\geq 40\\
    15x_1+2x_2\leq 60
\end{cases}$$
Al aplicar las propiedades:
$$SA\begin{cases}
    5x_1+15x_2+A_1-H_1= 50\\
    20x_1+5x_2+A_2-H_2= 40\\
    15x_1+2x_2+H_3= 60
\end{cases}$$

In [ ]:
sim=['x1','x2','a1','a2','h1','h2','h3']
H=['a1','a2','h3']
M=1e2
K=[[5,15,1,0,-1,0,0,50],
   [20,5,0,1,0,-1,0,40],
   [15,2,0,0,0,0,1,60],
   [25*M-4,20*M-2,0,0,-M,-M,0,90*M]]
MetodoSimplex.solve(K,sim,H)

Minimizar: $z=60x_1+50x_2$
$$SA\begin{cases}
    5x_1+3x_2\geq60\\
    2x_1+2x_2\geq 30\\
    7x_1+9x_2\geq 126
\end{cases}$$
Aplicando las propiedades:
$$SA\begin{cases}
    5x_1+3x_2+A_1-H_1=60\\
    2x_1+2x_2+A_2-H_2=30\\
    7x_1+9x_2+A_3-H_3=126
\end{cases}$$
$$z+x_1(14M-60)+x_2(14M-50)-H_1M-H_2M-H_3M=216M$$

In [ ]:
sim=['x1','x2','a1','a2','a_3','h1','h2','h3']
H=['a1','a2','a3']
M=1e3
K=[[5,3,1,0,0,-1,0,0,60],
   [2,2,0,1,0,0,-1,0,30],
   [7,9,0,0,1,0,0,-1,126],
   [14*M-60,14*M-50,0,0,0,-M,-M,-M,216*M]]
MetodoSimplex.solve(K,sim,H)

In [ ]:
class MetodoSimplex:
  #------------------------Método simplex base---------------------------
  def dividirfila(a,b):
    return sp.Matrix([a[i]/b[i] for i in range(len(a))])
  def mulfila(a,b):
    return sp.Matrix([a[i]*b[i] for i in range(len(a))])
  def solve(M,sim,H):
    M=sp.Matrix(M)
    #Selecciona el pivote
    ve=np.argmax((M.subs(sp.symbols('M'),1e4))[-1,:-1])
    vs=np.argmin([(i if i>0 else np.inf) for i in MetodoSimplex.dividirfila(M[:-1,-1],M[:-1,ve])])
    #Hace el remplazo de la variable
    H[vs]=sim[ve]
    M[vs,:]=M[vs,:]/M[vs,ve]
    #Hace ceros en la columna del pivote
    for i in range(M.shape[0]):
      #Realiza el procedimiento solo si la fila no es la del pivote
      if i!=vs:
        M[i,:]=M[i,:]*M[vs,ve]-M[vs,:]*M[i,ve]

    print("\n\n")
    display_latex(Latex(sp.latex(M)))
    print("\n\n")
    #Verifica si hay numeros positivos en la ultima fila exceptuando el resultado en p
    if len(np.array((M.subs(sp.symbols('M'),1e4))[-1,:-1],dtype="float64")[np.array((M.subs(sp.symbols('M'),1e4))[-1,:-1],dtype="float64")>0])>0:
      #Si hay positivos mayores a cero llama a la funcion pero con la nueva matriz
      return MetodoSimplex.solve(M,sim,H)
    #a grega a H un valor resultado para devolver un diccionario con los valores
    H.append("Resultado")
    M[-1,-1]=abs(M[-1,-1])
    return {H[i]: M[i,-1] for i in range(len(H))}

In [ ]:
sim='x_1 x_2 h_1 h_2 A_1 A_2'.split(" ")
H='A_1 A_2 h_2'.split(' ')

p=sp.symbols('p')
M=sp.symbols('M')
K=[[3,1,0,0,1,0,3],
   [4,3,-1,0,0,1,6],
   [1,2,0,1,0,0,4],
   [7*M-8,4*M-2,-M,0,0,0,9*M]]

display_latex(Latex(sp.latex(sp.Matrix(K))))
MetodoSimplex.solve(K,sim,H)